In [1]:
import os 
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

### Import modules

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, GlobalAveragePooling1D, Input
from tensorflow.keras import Model

### Tpy data generation

In [3]:
sentences = ["The movie was incredibly enjoyable.", # Positive
            "The dinner tasted fantastic.",
            "I love spending time with my friends.",
            "The team did a great job on the project.",
            "This is the best vacation ever.",
            "The concert was absolutely amazing.",
            "Her performance was outstanding.",
            "I can't wait for the next episode.",
            "The book was a captivating read.",
            "This app is very user-friendly.",
             
            "I had a terrible experience at the restaurant.", # Negative
            "The movie was a complete waste of time.",
            "This hotel is dirty and uncomfortable.",
            "The customer service was extremely disappointing.",
            "The product didn't work as advertised.",
            "I'm not a fan of this band.",
            "The flight was delayed and disorganized.",
            "The plot of the book was confusing and dull.",
            "I don't like this new app update.",
            "The project was poorly managed."]

labels = np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


#### Hyperparameters

In [4]:
vocab_size = 1000
embedding_dim = 10
max_length = 8
trunc_type = 'post' # pre vs post
padding_type = 'post' # pre vs post


#### Tokenization and Padding

In [5]:
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(sentences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

#### Check padding results

In [6]:
padded_sequences

array([[ 1,  9,  2, 14, 15,  0,  0,  0],
       [ 1, 16, 17, 18,  0,  0,  0,  0],
       [ 5, 19, 20, 10, 21, 22, 23,  0],
       [ 1, 24, 25,  3, 26, 27, 28,  1],
       [ 4,  6,  1, 29, 30, 31,  0,  0],
       [ 1, 32,  2, 33, 34,  0,  0,  0],
       [35, 36,  2, 37,  0,  0,  0,  0],
       [ 5, 38, 39, 40,  1, 41, 42,  0],
       [ 1, 12,  2,  3, 43, 44,  0,  0],
       [ 4, 13,  6, 45, 46, 47,  0,  0],
       [ 5, 48,  3, 49, 50, 51,  1, 52],
       [ 1,  9,  2,  3, 53, 54,  7, 10],
       [ 4, 55,  6, 56,  8, 57,  0,  0],
       [ 1, 58, 59,  2, 60, 61,  0,  0],
       [ 1, 62, 63, 64, 65, 66,  0,  0],
       [67, 68,  3, 69,  7,  4, 70,  0],
       [ 1, 71,  2, 72,  8, 73,  0,  0],
       [ 1, 74,  7,  1, 12,  2, 75,  8],
       [ 5, 77, 78,  4, 79, 13, 80,  0],
       [ 1, 11,  2, 81, 82,  0,  0,  0]])

### Model architecture

In [7]:
InpL = Input(shape=(max_length,))
EmbeddL = Embedding(vocab_size, embedding_dim, input_length=max_length)(InpL)
GAPL = GlobalAveragePooling1D()(EmbeddL)
DenL = Dense(16, activation='relu')(GAPL)
Output = Dense(1, activation='sigmoid')(DenL)

SentiModel = Model(InpL, Output)
SentiModel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['AUC'])

In [8]:
SentiModel.fit(x=padded_sequences, y=labels, epochs=100, verbose=1, shuffle=True)

Epoch 1/100
1/1 [==============================] - 0s 407ms/step - loss: 0.6924 - auc: 0.6000
Epoch 2/100
1/1 [==============================] - 0s 998us/step - loss: 0.6915 - auc: 0.6500
Epoch 3/100
1/1 [==============================] - 0s 996us/step - loss: 0.6905 - auc: 0.8000
Epoch 4/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6895 - auc: 0.8500
Epoch 5/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6885 - auc: 0.8650
Epoch 6/100
1/1 [==============================] - 0s 0s/step - loss: 0.6875 - auc: 0.9100
Epoch 7/100
1/1 [==============================] - 0s 0s/step - loss: 0.6865 - auc: 0.9200
Epoch 8/100
1/1 [==============================] - 0s 1ms/step - loss: 0.6855 - auc: 0.9200
Epoch 9/100
1/1 [==============================] - 0s 998us/step - loss: 0.6844 - auc: 0.9400
Epoch 10/100
1/1 [==============================] - 0s 998us/step - loss: 0.6834 - auc: 0.9700
Epoch 11/100
1/1 [==============================] - 0s 998us/step - los

### Input new sentence to predict the outcome

In [11]:
New_sentences = ["The movie was dull, disappointing, and terrible."]
New_sequences = tokenizer.texts_to_sequences(New_sentences)
New_padded_sequences = pad_sequences(New_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
New_padded_sequences

array([[ 1,  9,  2, 76, 61,  8, 49,  0]])

#### Make a prediction

In [13]:
SentiModel(New_padded_sequences)

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.43085372]], dtype=float32)>